In [18]:
import json
from collections import Counter  
import ast

# Enter your keys/secrets as strings in the following fields
credentials = {}  
credentials['CONSUMER_KEY'] = 'xf96zGbBtmbXnIJBIKbhg7IQ1'
credentials['CONSUMER_SECRET'] = 'HyHYCA8gB1oGN9H5bMEotatc3AfSAMSCK2SeoFVuTdYVwxTpgY'
credentials['ACCESS_TOKEN'] = '106489559-rxeRE7aH8f9Z7V50CsHfaYeZ7DrzmaJBAmn8miNi'
credentials['ACCESS_SECRET'] = 'lJbQlDcLjOtbZNSyJ1qAVJ7jvBpBEJOHnAOoI1KPaetJj'

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:  
    json.dump(credentials, file)

In [9]:
from twython import TwythonStreamer  
import csv

# Filter out unwanted data
def process_tweet(tweet):  
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['id'] = tweet['user']['id']
    d['user_loc'] = tweet['user']['location']
    d['followers']= tweet['user']['followers_count']
    return d


# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open(r'saved_tweets_refugees_definitive.csv', 'a') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

In [10]:
# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'],  
                    creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
# Start the stream
stream.statuses.filter(track='refugees')

ChunkedEncodingError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))

In [14]:
import pandas as pd  
headers = ['hashtags', 'tweet', 'user_name', 'id','location', 'followers']
tweets_2 = pd.read_csv("saved_tweets_refugees_definitive.csv", header = None, names = headers)  
tweets_2.head() 

,hashtags,tweet,user_name,id,location,followers
0,[],RT @charliekirk11: Did you know:\n\nThe govern...,cyelley,134876784,"California, USA",463
1,[],RT @charliekirk11: Did you know:\n\nThe govern...,metamancer,121989966,NaN,71
2,[],RT @5gBxizlCqzFrc3K: @Refugees @RefugeesChief ...,balkees51046223,1081605573383282691,Türkiye,202
3,[],RT @charliekirk11: Did you know:\n\nThe govern...,DoeAnon66,3164507364,PNW,209
4,[],RT @charliekirk11: Did you know:\n\nThe govern...,MyPrezTweets,758839025645989889,Earth,2618


In [15]:
tweets_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9370 entries, 0 to 9369
Data columns (total 6 columns):
hashtags     9370 non-null object
tweet        9370 non-null object
user_name    9370 non-null object
id           9370 non-null int64
location     5730 non-null object
followers    9370 non-null int64
dtypes: int64(2), object(4)
memory usage: 439.3+ KB


### Getting the locations

In [19]:
list_location_strings = [entry for entry in tweets.location] 

In [20]:
counter_locations = Counter(list_location_strings)

In [21]:
counter_locations.most_common(250)

[(nan, 3640),
 ('United States', 326),
 ('USA', 118),
 ('Texas, USA', 105),
 ('California, USA', 102),
 ('Florida, USA', 95),
 ('Sinop, Türkiye', 66),
 ('North Carolina, USA', 42),
 ('Los Angeles, CA', 42),
 ('Washington, DC', 40),
 ('Chicago, IL', 38),
 ('Michigan, USA', 34),
 ('Texas', 33),
 ('Houston, TX', 32),
 ('Karabük, Türkiye', 32),
 ('Arizona, USA', 31),
 ('Georgia, USA', 30),
 ('Colorado, USA', 29),
 ('New Jersey, USA', 27),
 ('Canada', 27),
 ('Pennsylvania, USA', 27),
 ('New York, USA', 26),
 ('Virginia, USA', 25),
 ('Ohio, USA', 25),
 ('New York', 25),
 ('Illinois, USA', 25),
 ('Tennessee, USA', 24),
 ('Washington, USA', 24),
 ('Las Vegas, NV', 24),
 ('California', 22),
 ('New York, NY', 21),
 ('Indiana, USA', 21),
 ('Massachusetts, USA', 19),
 ('Earth', 18),
 ('London, England', 18),
 ('Louisiana, USA', 18),
 ('Austin, TX', 17),
 ('Oregon, USA', 17),
 ('Wisconsin, USA', 17),
 ('San Diego, CA', 16),
 ('Pittsburgh, PA', 16),
 ('Portland, OR', 16),
 ('المان', 16),
 ('Ontario,

In [22]:
df_location_2 = pd.DataFrame.from_dict(counter_locations, orient='index').reset_index()
df_location_2 = df_location_2.rename(columns={'index':'locations', 0:'count'})
df_location_2.head()

,locations,count
0,"California, USA",102
1,NaN,3640
2,Türkiye,9
3,PNW,2
4,Earth,18


In [24]:
df_location_2 = df_location_2.sort_values(['count'], ascending=False)

In [28]:
df_loc_filter_2 = df_location_2[df_location_2['count'] > 10]

In [39]:
df_loc_filter_2 = df_loc_filter_2.dropna(axis=0)

df_loc_filter_2.head()

,locations,count
19,United States,326
18,USA,118
27,"Texas, USA",105
0,"California, USA",102
7,"Florida, USA",95


## Getting the hashtags

In [30]:
# Extract hashtags and put them in a list
list_hashtag_strings = [entry for entry in tweets_2.hashtags]  
list_hashtag_lists = ast.literal_eval(','.join(list_hashtag_strings))  
hashtag_list = [ht.lower() for list_ in list_hashtag_lists for ht in list_]

# Count most common hashtags
counter_hashtags = Counter(hashtag_list)  
counter_hashtags.most_common(50) 

[('ice', 77),
 ('racistpresident', 45),
 ('refugees', 37),
 ('saveali', 29),
 ('delaram', 27),
 ('iranianrefugeesinturkey', 23),
 ('iran', 19),
 ('trudeau', 18),
 ('iranian', 17),
 ('eu', 14),
 ('standwithlraqirefugees', 10),
 ('resettlement4iranian', 10),
 ('migrants', 10),
 ('libya', 10),
 ('standwithiraqirefugees', 9),
 ('thesquad', 9),
 ('canada', 8),
 ('forrefugees', 7),
 ('rohingya', 7),
 ('homeless', 6),
 ('humanrights', 6),
 ('asylum', 6),
 ('iranianrefugeeinturkey', 6),
 ('us', 6),
 ('resettlement', 6),
 ('americafirst', 5),
 ('syria', 5),
 ('mediterranean', 5),
 ('socialsecurity', 5),
 ('migration', 4),
 ('boycottamazon', 4),
 ('boycottprimeday', 4),
 ('ep', 4),
 ('greece', 4),
 ('yesdearleader', 4),
 ('turkey', 3),
 ('bangladesh', 3),
 ('internallydisplacedpersons', 3),
 ('presidenttrump', 3),
 ('williambila', 3),
 ('nosharia', 3),
 ('refugee', 3),
 ('iom', 3),
 ('women', 2),
 ('mmc', 2),
 ('withrefugees', 2),
 ('refugeeswelcome', 2),
 ('lgbt', 2),
 ('pakistan', 2),
 ('chris

In [33]:
df_hashtags = pd.DataFrame.from_dict(counter_hashtags, orient='index').reset_index()
df_hashtags = df_hashtags.rename(columns={'index':'hashtags', 0:'count'})
df_hashtags.head()

,hashtags,count
0,standwithlraqirefugees,10
1,trudeau,18
2,women,2
3,children,1
4,refugees,37


In [34]:
df_hashtags = df_hashtags.sort_values(['count'], ascending=False)

In [36]:
df_hashtags = df_hashtags[df_hashtags['count'] > 30]

In [38]:
df_hashtags.head()

,hashtags,count
145,ice,77
56,racistpresident,45
4,refugees,37


# User with most twits and followers

In [55]:
df_users_followers = tweets_2.loc[:, ['user_name', 'id', 'followers','tweet']]

In [56]:
df_users_followers.head()

,user_name,id,followers,tweet
0,cyelley,134876784,463,RT @charliekirk11: Did you know:\n\nThe govern...
1,metamancer,121989966,71,RT @charliekirk11: Did you know:\n\nThe govern...
2,balkees51046223,1081605573383282691,202,RT @5gBxizlCqzFrc3K: @Refugees @RefugeesChief ...
3,DoeAnon66,3164507364,209,RT @charliekirk11: Did you know:\n\nThe govern...
4,MyPrezTweets,758839025645989889,2618,RT @charliekirk11: Did you know:\n\nThe govern...


In [59]:
df_users_followers.groupby(['user_name']).

,id,followers
user_name,,
0123missy,8.910378e+07,131.0
05Zine,4.085902e+08,448.0
0rangeMoose,1.210854e+09,27.0
104Crawford,4.228800e+09,4.0
10Harris76,2.585276e+08,30.0
1165R_USPatriot,3.578206e+08,1625.0
11XMaTriX,7.902069e+17,291.0
1200rGSrider,8.134842e+07,13220.0
123RHGreen,3.693456e+07,353.0
